In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex 客户端库：用于在线预测的本地文本二元分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_local_text_binary_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_local_text_binary_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用Python的Vertex客户端库部署一个在本地训练的自定义文本二分类模型，以进行在线预测。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[IMDB电影评论](https://www.tensorflow.org/datasets/catalog/imdb_reviews)。在本教程中使用的数据集版本存储在一个公共云存储桶中。训练好的模型预测评论的情感是积极的还是消极的。

### 目标

在这个笔记本中，您将在笔记本中创建一个自定义模型，然后学习将本地训练的模型部署到 Vertex，然后对部署的模型进行预测。您还可以使用`gcloud`命令行工具或在线使用Google Cloud控制台创建和部署模型。

执行的步骤包括：

- 本地创建一个模型。
- 本地训练模型。
- 查看模型评估。
- 将模型上传为Vertex`Model`资源。
- 将`Model`资源部署到一个服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

### 费用

此教程使用谷歌云（GCP）的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预计使用情况生成成本估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦安装了 Vertex 客户端库和 Google 云存储，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有的话，确保你在 GPU 运行时中运行这个笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置你的谷歌云项目

**无论你使用哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建一个账户时，你会获得 $300 的免费信用用于计算/存储成本。

2. [确保你的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经在谷歌云笔记本中安装。

5. 在下面的单元格中输入你的项目 ID。然后运行该单元格，确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 运行以 `!` 开头的行作为 shell 命令，并将以 `$` 开头的 Python 变量插值到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于本笔记本的其余操作。以下是Vertex支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不得使用多区域存储桶来训练Vertex。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持情况，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加直播教程会话，您可能会使用一个共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并附加在将在此教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的 Google Cloud 账号

**如果您正在使用 Google Cloud Notebook**，您的环境已经通过身份验证。跳过此步骤。

**如果您正在使用 Colab**，运行下面的单元格，并按照提示进行身份验证，通过 oAuth 认证您的账号。

**否则**，请按照以下步骤操作：

在 Cloud Console 中，前往 [创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务账号**。

在 **服务账号名称** 栏中输入一个名称，并点击 **创建**。

在 **将此服务账号授权给项目** 部分，点击角色下拉列表。在筛选框中输入 "Vertex"，并选择 **Vertex 管理员**。在筛选框中输入 "Storage Object Admin"，并选择 **Storage Object Admin**。

点击创建。一个包含您密钥的 JSON 文件将下载到您的本地环境。

在下面的单元格中将您的服务账号密钥路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪个笔记本环境，都需要按照以下步骤操作。**

当您使用 Vertex 客户端库提交一个自定义训练作业时，您需要将包含训练代码的 Python 包上传到一个云存储桶中。Vertex 将从这个包中运行代码。在本教程中，Vertex 还会将作业生成的训练模型保存在同一个存储桶中。然后，您可以根据这个输出创建一个 `Endpoint` 资源，以便进行在线预测。

在下面设置您的云存储桶的名称。存储桶的名称在所有 Google Cloud 项目中必须是全局唯一的，包括您组织外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容验证对云存储存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点 API 服务的数据集、模型、作业、流水线和端点服务端点。
- `PARENT`：数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

硬件加速器

设置硬件加速器（例如GPU）用于预测。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器镜像，以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器镜像，每个VM分配4个Nvidia Telsa K80 GPU，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以在CPU上运行容器镜像。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 容器（Docker）映像

接下来，我们将设置用于预测的Docker容器映像

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示了一些可用的预构建映像：

  - TensorFlow 1.15
    - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
  - TensorFlow 2.1
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
  - TensorFlow 2.2
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
  - TensorFlow 2.3
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
  - XGBoost
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
  - Scikit-learn
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

要获取最新列表，请参见[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为为用于预测的VM配置计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在，您已经准备好开始本地训练一个自定义模型IMDB电影评论，然后部署该模型到云端。

## 设置客户端

Vertex客户端库采用客户端/服务器模型。在您的一侧（Python脚本）中，您将创建一个客户端，该客户端将向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前配置它们。

- 用于“模型”资源的模型服务。
- 用于部署的端点服务。
- 提供预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 在本教程中，您将在本地训练一个IMDB电影评论模型。

### 将位置设置为存储训练模型

您设置变量`MODEL_DIR`来指定在您的云存储桶中保存TensorFlow SavedModel格式的模型的位置。

此外，您为训练脚本创建一个本地文件夹。

In [ ]:
MODEL_DIR = BUCKET_NAME + "/imdb"
model_path_to_deploy = MODEL_DIR

! rm -rf custom
! mkdir custom
! mkdir custom/trainer

#### Task.py 内容

在下一个单元格中，您可以写入训练脚本 task.py 的内容。我不会详细说明，只是让您浏览一下。总结如下：

- 从命令行获取保存模型工件的目录（`--model_dir`），如果没有指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets (tfds) 加载 IMDB 电影评论数据集。
- 使用 TF.Keras 模型 API 构建一个简单的 RNN 模型。
- 编译模型 (`compile()`)。
- 根据参数 `args.distribute` 设置训练分发策略。
- 使用指定的 epoch 训练模型 (`fit()`)。
- 将训练后的模型保存到指定的模型目录 (`save(args.model_dir)`)。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for IMDB

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=1e-4, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print(device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets():
    dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
    train_dataset, test_dataset = dataset['train'], dataset['test']
    encoder = info.features['text'].encoder

    padded_shapes = ([None],())
    return train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes), encoder


train_dataset, encoder = make_datasets()

# Build the Keras model
def build_and_compile_rnn_model(encoder):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(encoder.vocab_size, 64),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(args.lr),
        metrics=['accuracy'])
    return model


with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_rnn_model(encoder)

# Train the model
model.fit(train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

训练模型

In [ ]:
! python custom/trainer/task.py --epochs=10 --model-dir=$MODEL_DIR

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶加载它，然后您可以执行一些操作，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径传递给它，路径由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们找出模型的表现有多好。

### 加载评估数据

您将使用 `tfds.datasets` 中的 `load()` 方法加载 IMDB 电影评论测试（保留集）数据。这将返回一个包含两个元素的元组。第一个元素是数据集，第二个是数据集的信息，其中包含预定义的词汇编码器。编码器将把单词转换为数字嵌入，这些嵌入是预先训练过的，并在自定义训练脚本中使用。

在训练模型时，您需要为文本设置一个固定的输入长度。对于前向传递批处理，相应的 `tf.dataset` 的 `padded_batch()` 属性被设置为将每个输入序列填充到相同的形状以用于一个批次。

对于测试数据，您还需要相应地设置 `padded_batch()` 属性。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]
encoder = info.features["text"].encoder

BATCH_SIZE = 64
padded_shapes = ([None], ())
test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes)

### 进行模型评估

现在评估自定义工作中模型的表现如何。

In [ ]:
model.evaluate(test_dataset)

## 上传用于服务的模型

接下来，您将把 TF.Keras 模型从自定义作业上传到 Vertex `Model` 服务，该服务将为您的自定义模型创建一个 Vertex `Model` 资源。在上传过程中，您需要定义一个服务函数，将数据转换为模型所需的格式。如果您发送编码数据到 Vertex，则您的服务函数将确保在数据传递到模型之前在模型服务器上对数据进行解码。

### 服务函数如何工作

当您发送请求到在线预测服务器时，请求将被 HTTP 服务器接收。HTTP 服务器从 HTTP 请求内容中提取预测请求。提取的预测请求将转发到服务函数。对于 Google 预置预测容器，请求内容作为 `tf.string` 传递给服务函数。

服务函数由两部分组成：

- `预处理函数`:
  - 将输入（`tf.string`）转换为底层模型的输入形状和数据类型（动态图）。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如，归一化、缩放等。
- `后处理函数`:
  - 将模型输出转换为接收应用程序预期的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，制作 JSON 对象等。

预处理和后处理函数都被转换为与模型融合的静态图。来自底层模型的输出传递给后处理函数。后处理函数将转换/打包后的输出传递回 HTTP 服务器。HTTP 服务器将输出作为 HTTP 响应内容返回。

在为 TF.Keras 模型构建服务函数时，您需要考虑一个问题，即它们将作为静态图运行。这意味着您不能使用需要动态图的 TF 图操作。如果您这样做，编译服务函数时将会出现错误，并指出您正在使用不受支持的 EagerTensor。

获取服务功能签名

通过将模型重新加载到内存中，并查询与每个层对应的签名，您可以获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称，以便在以后进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用此辅助函数`upload_model`来上传您的模型，存储在SavedModel格式中，到`Model`服务，该服务将为您的模型实例化一个Vertex `Model`资源实例。一旦您完成了这个步骤，您可以像任何其他Vertex `Model`资源实例一样使用`Model`资源实例，例如部署到用于提供预测的`Endpoint`资源。

辅助函数接受以下参数：

- `display_name`：`Endpoint`服务的可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：我们的SavedModel工件的Cloud Storage路径。对于本教程，这是我们在变量`MODEL_DIR`中指定的`trainer/task.py`保存模型工件的Cloud Storage位置。

辅助函数调用`Model`客户服务的`upload_model`方法，该方法接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex根路径。
- `model`：Vertex `Model`资源实例的规范。

现在让我们深入了解Vertex模型规范`model`。这是一个包含以下字段的字典对象：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex `Dataset`资源的情况下构建的，您将留空（`''`）。
- `artificat_uri`：SavedModel格式中存储模型的Cloud Storage路径。
- `container_spec`：这是将安装在`Endpoint`资源上的Docker容器的规范，从中`Model`资源将提供预测。使用您之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源将返回一个长时间运行的操作，因为这可能需要一些时间。您可以调用response.result()，这是一个同步调用，当Vertex Model资源准备就绪时将返回。

辅助函数返回对应的Vertex Model实例`upload_model_response.model`的Vertex完全合格的标识符。您将在变量`model_to_deploy_id`中保存该标识符以便后续步骤使用。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model("imdb-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy)

### 获取`Model`资源信息

现在让我们获取只属于您的模型的模型信息。使用此辅助函数`get_model`，并传入以下参数：

- `name`：`Model`资源的 Vertex 唯一标识符。

这个辅助函数调用 Vertex `Model`客户服务的方法`get_model`，并传入以下参数：

- `name`：`Model`资源的 Vertex 唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

## 部署`Model`资源

现在部署经过训练的Vertex自定义`Model`资源。这需要两个步骤：

1. 为部署`Model`资源创建一个`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

### 创建一个 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个用于部署模型以提供预测的终端节点，具有以下参数:

- `display_name`: 用于 `Endpoint` 资源的可读名称。

这个辅助函数使用终端节点客户端服务的 `create_endpoint` 方法，该方法接受以下参数:

- `display_name`: 用于 `Endpoint` 资源的可读名称。

创建一个 `Endpoint` 资源会返回一个长时间运行的操作，因为为服务而配置 `Endpoint` 资源可能需要一些时间。您可以调用 `response.result()`，这是一个同步调用，当终端节点资源准备就绪时会返回。该辅助函数返回 `Endpoint` 资源的 Vertex 完全限定标识符：`response.name`.

In [ ]:
ENDPOINT_NAME = "imdb_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“终端点”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理您的在线预测请求时，您有几种扩展计算实例的选择：

- 单个实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：在线预测请求被分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数。当模型首次部署到实例时，将为固定数量的计算实例配置和在线预测请求均匀分配到它们之间。

- 自动扩展：在线预测请求被分配到可伸缩数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）数量设置为模型首次部署和取消部署时要配置的计算实例数，并将最大（`MAX_NODES`）数量设置为根据负载条件要配置的计算实例数。

计算实例的最小数量对应于字段 `min_replica_count`，而最大数量对应于字段 `max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署`Model`资源到`Endpoint`资源

使用这个辅助函数`deploy_model`将`Model`资源部署到为提供预测而创建的`Endpoint`资源，使用以下参数：

- `model`：要从训练流水线上传（部署）的Vertex完全限定模型标识符。
- `deploy_model_display_name`：部署模型的人可读名称。
- `endpoint`：要部署模型到的Vertex完全限定终端标识符。

辅助函数调用`Endpoint`客户端服务的`deploy_model`方法，该方法接受以下参数：

- `endpoint`：要将`Model`资源部署到的Vertex完全限定`Endpoint`资源标识符。
- `deployed_model`：部署模型的要求规范。
- `traffic_split`：流向此模型的端点流量的百分比，以字典形式指定一个或多个键/值对。
   - 如果只有一个模型，则指定为**{"0": 100}**，其中"0"指代上传的此模型，100表示100%的流量。
   - 如果终端上存在现有模型，流量将被分配到这些模型，使用`model_id`指定为**{"0": percent, model_id: percent, ...}**，其中`model_id`是要部署到的终端的现有模型的模型ID。百分比必须相加为100。

现在让我们更深入`deployed_model`参数。此参数指定为Python字典，具有最少必需字段：

- `model`：部署的（上传）模型的Vertex完全限定模型标识符。
- `display_name`：部署模型的人可读名称。
- `disable_container_logging`：这会禁用容器事件的日志记录，如执行失败（默认情况下容器日志记录已启用）。容器日志记录通常在部署调试时启用，然后在投入生产时禁用。
- `dedicated_resources`：指定为为提供预测请求缩放的计算实例（副本）数量。
  - `machine_spec`：要提供的计算实例。使用您先前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。
  - `min_replica_count`：初始提供的计算实例数，您之前设置为变量`MIN_NODES`。
  - `max_replica_count`：要扩展到的最大计算实例数，您之前设置为变量`MAX_NODES`。

#### 流量分割

现在让我们更深入`traffic_split`参数。此参数指定为Python字典。一开始可能有点混乱。让我解释一下，您可以将模型的多个实例部署到终端，并设置每个实例的流量百分比。

为什么要这样做？也许您已经在生产中部署了以前的版本--让我们称之为v1。您在v2上获得了更好的模型评估，但在您部署到生产环境之前，您无法确切知道它是否真的更好。因此，在流量分割的情况下，您可能希望将v2部署到与v1相同的终端，但它只获得例如10%的流量。这样，您可以在不干扰大多数用户的情况下监控其表现，直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作`response`。我们将通过调用`response.result()`同步等待操作完成，这将阻塞直到模型部署完毕。如果这是第一次将模型部署到终端，则可能需要额外几分钟来完成资源的提供。

In [ ]:
DEPLOYED_NAME = "imdb_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

提交一个在线预测请求

现在对您部署的模型进行在线预测。

### 准备请求内容

由于数据集是一个`tf.dataset`，它充当生成器的角色，我们必须将其用作迭代器以访问测试数据中的数据项目。我们执行以下操作以从测试数据中获取单个数据项：

- 使用`take(1)`方法将每次迭代要绘制的批次数属性设置为一。
- 通过测试数据迭代一次--即，在for循环中执行中断。
- 在单次迭代中，我们保存以元组形式表示的数据项。
- 数据项将是元组的第一个元素，然后您将从张量转换为numpy数组--`data[0].numpy()`。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]

test_dataset.take(1)
for data in test_dataset:
    print(data)
    break

test_item = data[0].numpy()

### 发送预测请求

好的，现在你有一个测试数据项目。使用这个辅助函数`predict_data`，它接受以下参数：

- `data`：测试数据项目是一个64填充的 numpy 1D 数组。
- `endpoint`：模型部署的 Vertex AI 完全合格标识符。
- `parameters_dict`：用于提供服务的额外参数。

此函数使用预测客户端服务，并使用以下参数调用`predict`方法：

- `endpoint`：模型部署的 Vertex AI 完全合格标识符。
- `instances`：要预测的实例（数据项目）列表。
- `parameters`：用于提供服务的额外参数。

要将测试数据传递给预测服务，必须将其打包以便传输到服务二进制文件，如下所示：

    1. 将数据项目从1D numpy数组转换为1D Python列表。
    2. 将预测请求转换为序列化的 Google protobuf (`json_format.ParseDict()`)

预测请求中的每个实例都是以下形式的字典条目：

                        {input_name: content}

- `input_name`：底层模型的输入层的名称。
- `content`：数据项目作为1D Python列表。

由于`predict()`服务可以接受多个数据项目（实例），你将把单个数据项目发送为一个数据项目列表。最后一步是将实例列表打包成 Google 的 protobuf 格式 -- 这是我们传递给`predict()`服务的内容。

`response`对象返回一个列表，列表中的每个元素对应请求中的相应图像。在每个预测的输出中，你将看到：

- `predictions` -- 预测的二进制情感在0（负面）到1（正面）之间。

In [ ]:
def predict_data(data, endpoint, parameters_dict):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: data.tolist()}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_data(test_item, endpoint_id, None)

## 卸载 `Model` 资源

现在从正在提供服务的 `Endpoint` 资源中卸载您的 `Model` 资源。请使用这个辅助函数 `undeploy_model`，它需要以下参数：

- `deployed_model_id`：当 `Model` 资源部署到 `Endpoint` 服务时端点服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 Vertex 完全限定标识符。

这个函数调用端点客户端服务的 `undeploy_model` 方法，具有以下参数：

- `deployed_model_id`：当 `Model` 资源部署时端点服务返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 Vertex 完全限定的 `Endpoint` 资源标识符。
- `traffic_split`：在 `Endpoint` 资源上的其余部署模型之间如何分配流量。

由于这是 `Endpoint` 资源上唯一部署的模型，您可以通过将 `traffic_split` 设置为 {} 来简单地保留为空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理工作

要清理掉本项目中使用的所有GCP资源，您可以[删除用于本教程的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME